In [ ]:
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

sys.path.append('./../')  # Needed for importing from simulation_utils.py
from simulation_utils import *

# Define BBPSSW circuit (Alice and Bob each apply CNOT: control=first qubit, target=second qubit)
bbpssw_circuit = QuantumCircuit([
    Gate(kron([CNOT(), CNOT()])        # Apply CNOT(1A, 2A) and CNOT(1B, 2B)
])

# Construct POVM for the 2nd EPR pair (qubits 2A and 2B)
povm_epr2 = POVM(
    nqubits=4,
    meas_qubits=[2, 3],
    partial_trace=True,
)

numpy_sim_results = []

for epr_channel_fidelity in np.linspace(0, 1, 11):
    # Create two Werner states representing noisy EPR pairs
    epr_1 = epr_2 = werner_state(epr_channel_fidelity)

    # Create combined system (qubits in order: |1A, 1B, 2A, 2B>)
    rho = kron([epr_1, epr_2])

    # Apply BBPSSW circuit
    rho = bbpssw_circuit(rho)

    # Compute probabilities and outcome states when measuring qubits 2A and 2B
    probs, rho_outs = povm_epr2(rho)

    # Compute fidelity wrt target Bell state phi_00
    target_state = bell_state(0, 0)
    fidels = np.array([fidelity(dm, target_state) for dm in rho_outs])

    # Compute probability of success and average fidelity given success
    prob_succ = probs[0] + probs[-1]
    fidel_succ = (fidels[0] * probs[0] + fidels[-1] * probs[-1]) / prob_succ if prob_succ else 0

    numpy_sim_results.append([epr_channel_fidelity, prob_succ, fidel_succ])

numpy_sim_results = pd.DataFrame(numpy_sim_results,
                                columns=['EPR channel fidelity', 'P_Succ', 'F_succ'])

# Theoretical functions for BBPSSW
def prob_succ_theory(F: float):
    """Theoretical success probability for BBPSSW protocol"""
    return F**2 + (1-F)**2/9 + 2*F*(1-F)/3

def fidelity_succ_theory(F: float):
    """Theoretical fidelity after successful BBPSSW iteration"""
    numerator = F**2 + (1-F)**2/9
    denominator = F**2 + 2*F*(1-F)/3 + 5*(1-F)**2/9
    return numerator / denominator

# Plotting code
ps = np.linspace(0, 1, 101)
plt.figure(figsize=(10, 6))

# Theory curves
plt.plot(ps, [prob_succ_theory(p) for p in ps], 'c-', label='Theory')
plt.plot(ps, [fidelity_succ_theory(p) for p in ps], 'm-', label='Theory')

# Numpy simulation results
plt.plot(numpy_sim_results['EPR channel fidelity'], numpy_sim_results['P_Succ'],
         'xb', label='Numpy P_succ')
plt.plot(numpy_sim_results['EPR channel fidelity'], numpy_sim_results['F_succ'],
         'xg', label='Numpy F_succ')

# Formatting
plt.title('BBPSSW Protocol Performance')
plt.xlabel('EPR Channel Fidelity')
plt.ylabel('Probability/Fidelity')
plt.xticks(np.arange(0, 1.1, 0.1))
plt.legend()
plt.grid(True)
plt.show()